In [1]:
import os
import numpy as np
import tensorflow as tf
import pydub

In [2]:
labels = []
for file in os.listdir("../dataset"):
    labels.append(file.split('.')[0])

In [3]:
print(labels)

['Abdollah', 'Azra', 'Davood', 'Javad', 'Khadijeh', 'Kiana', 'Mahdi', 'Maryam', 'Matin', 'Melika', 'Mohadeseh', 'Mohammad', 'Mohammad_parvari', 'Mona', 'Nahid', 'Nima', 'Omid', 'Parisa', 'Parsa', 'Sajedeh', 'Shima', 'Tara']


preprocess audio

In [8]:
file_path = "input_audio\Mahdi_2.ogg"
audio = pydub.AudioSegment.from_file(file_path)

audio = audio.set_sample_width(2)    # convert to 16-bit (2 bytes per sample) to avoid tensorflow error
audio = audio.set_frame_rate(48000)
audio = audio.set_channels(1)        # convert stereo audio to mono audio to avoid tensorflow error

chunks = pydub.silence.split_on_silence(audio, min_silence_len=2000, silence_thresh=-45)
result = sum(chunks)
result.export("input_audio\input_audio.wav", format="wav")

<_io.BufferedRandom name='input_audio\\input_audio.wav'>

turn input audio into 1 second chunks

In [12]:
audio = pydub.AudioSegment.from_file("input_audio\input_audio.wav")
chunks = pydub.utils.make_chunks(audio, 1000)

os.makedirs("input_audio_chunks")
for i , chunk in enumerate(chunks):
    if len(chunk) < 1000 :
        continue
    chunk.export(os.path.join("input_audio_chunks", f"voice{i}.wav"), format="wav")

In [10]:
audio_chunks = []

for audio_chunk in os.listdir("input_audio_chunks"):
    audio_chunks.append(audio_chunk)
# audio_chunks

In [11]:
xs = []
for audio_chunk in audio_chunks:
    x = tf.io.read_file("input_audio_chunks/"+audio_chunk)
    x, sample_rate = tf.audio.decode_wav(x, desired_channels=1, desired_samples=48000)
    x = tf.squeeze(x, axis=-1)
    x = x[tf.newaxis,...]
    xs.append(x)

load model

In [14]:
model = tf.keras.models.load_model('../weights/audio_classification.h5')

predict

In [15]:
preds = []
for x in xs:
    pred = model(x)
    label = np.argmax(pred)
    preds.append(labels[label])
# print(preds)
most_common = max(set(preds), key=preds.count)                # Finds the most common element in preds list
# print(most)
print("The voice belongs to: ", most_common)

The voice belongs to:  Mahdi
